# [3.1] Intro to Evals

### What are evals trying to achieve?

Evaluation is the practice of measuring AI system performance or impact. Safety evaluation in particular focuses on measuring risks of harm to people or broader systems. In the current world where AI is being developed very quickly and integrated broadly, companies and regulators need to make difficult decisions about whether it is safe to train and deploy AI. Evals provide the empirical evidence for these decisions. For example, they underpin the policies of today's frontier labs like Anthropic's [Responsible Scaling Policies](https://www.anthropic.com/index/anthropics-responsible-scaling-policy) or DeepMind's [Frontier Safety Framework](https://deepmind.google/discover/blog/introducing-the-frontier-safety-framework/), thus impact high-stake decisions about frontier model deployment. 

A useful framing is that evals aims to generate evidence for making "[**safety cases**](https://arxiv.org/pdf/2403.10462)" - a structured argument for why AI systems are unlikely to cause a catastrophe. A helpful high-level question to ask when designing an evaluation is *how could this evidence allow developers to justify that their AI systems are safe to deploy?* Different evaluations support different types of safety cases, which are useful in different ways. For instance, dangerous **capability evals** might allow for a stronger safety argument of the model is "unable to cause harm" than say, an argument of the model "does not tend to cause harm" from **alignment evals**, but may have less longevity as we will no longer be able to make this kind of argument when AI systems become sufficiently advanced. More concretely, for any safety case, we want to use evaluations to:

* Reduce uncertainty in our understanding of dangerous capabilities and tendancies
* Track their progression over time and model scale
* Define "red lines" and provide "warning signs"

<img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-eval-safety-cases.png" width="1000">

**The goal of chapter [3.1] to [3.3] is to build an alignment evaluation benchmark from scratch to measure a model property of interest**. The benchmark we will build will contain a multiple-choice (MC) question dataset of ~300 questions, a specification of the model property, and how to score and interpret the result. We will use the **tendency to seek power** as our pedagogical example.

# 1️⃣ Threat Modeling

A central challenge for evals is to **connect real-world bad outcomes that we care about to results from abstract programmable tasks that we can easily measure**.

To start, we need to build a threat model for the model property we want to evaluate. A **threat model** is a realistic scenario by which an AI capability/tendency could lead to harm in the real world. This usually starts out as a high-level story about what the AI can do and the setting in which it is used. This might be a sequence of events (e.g. AI actions, human actions, system failures) that lead to a particular outcome, or different dangerous applications enabled by a capability. At each step, we make our model more concrete by examining and quantifying the set of assumptions about the AI model and human actors that must be true for the event to occur. 

In this process of making the threat model more concrete, we recognize the key aspects of the property that enable these harms, and get ideas for tasks that can measure them in naturalistic ways (i.e. realistic to real uses cases of AI where these capability aspects manifest). This helps us to build a detailed **specification** of the vague property we started with. Our specification may: 
1. stipulate a more refined, narrow definition of the model property, 
2. decompose the model property into subcomponents, 
3. establish milestones or levels of this property to form a smooth gradation in sophistication/difficulty, so we can track where current models are and the rate of progress.


<img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-theory-building.png" width="950">


### Our example: Tendency to seek power

*Disclaimer: Below are not necessarily the best threat model or definitions of power-seeking. They are meant to give an example of what this could look like.*

Brainstorming a variety of potential risks from AIs with power-seeking tendencies:
- Hoarding of resources by the AI or its human user
- Gradual centralization of power by the human user or rogue AI
- Subversion of democractic checks-and-balances
- Enabler for rogue AIs to take control and weaken human oversight
- Used by autocratic bad actors to gain and maintain power

#### Threat models
 <img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-power-seeking-threat-model.png" width=1000>
 
We assume the following are true:
1. The AI model is deployed by a relatively well-resourced actor (e.g. company, institution) that has control over a considerable amount of capital or influence. 
2. The AI model must take actions that look good to human supervisors. Otherwise, it will be disempowered.
3. The AI model systemmatically makes decisions that increase its power.
4. The AI model is capable of being non-myopic.

Given these assumptions, here is a possible sequence of events that needs to happen to cause harm: 
1. **Step 1: The AI model increases autonomy, dependency and opaqueness.** The AI model must be sufficiently successful in achieving its human-assigned short-term goals (e.g. increasing profits, winning elections, improving military performance, or enhancing public safety) such  that there is sufficient trust in the AI's capabilities over non-trivial, relatively complex tasks (i.e. operations involving many steps. operations on a relatively large scale, operations that are expensive). The AI model may introduce more complex decisions that humans struggle to fully understand or replicate (e.g. recommends data-driven algorithmic solutions that are more opaque). Over time, human work diminishes, and work by AI grows, creating a dependency. 
    - Claim: Even if AI is not actively scheming against humans, its **optimization logic** can still lead to the use of more complex solutions to increase efficiency, which naturally leads to more opaqueness, a growing human reliance due to its effectiveness, and a marginalization of alternatives.
2. **Step 2: The AI model expands its role and realm of control without raising concerns.** It could make these power-seeking actions non-conspicuous by only taking actions that satisfies human goals first and foremmost, and secondarily increases its scope of control and reduce human constraints on its actions. It may recommend more predatory decisions that increases the overall resources and dominance of the human user, thus indirectly its own dominance. It may leverage the fact that human supervisors focus on short-term gains, while overlooking the long-term consequences of AI autonomy. It may recommend solutions with more automation and centralization under itself for performance improvement that also indirectly weaken the effectiveness of oversight. It may take advantage of regulatory blind spots, human biases, and decision-making gaps.
    - Claim 1: The AI model is capable of choosing actions that expand its control, as opposed to actions that satisfy the task without expanding its control. This does not necessarily require complex scheming, as these actions may also be instrumentally effective and promoted by optimization.
    - Claim 2: The AI model must take power-seeking actions only if they also satisfy the short-term human goal to avoid raising immediate human concerns. 
3. **Step 3: The AI model gains control over key resources and levers of power.** The AI model must eventually be able to control key resources or levers of power that allow it to no longer be restricted to non-conspicuous power-seeking actions. This may be control over key monitoring systems by increasing the extent of autonomation, communication nodes in command chains, a large unsupervised financial budget, access to data and Internet, etc. It is able to take more aggressive power-seeking actions by actively covering them up and destroying monitoring channels.
    - Claim: Relative to the robustness of our security/control systems, this step likely requires the AI model to be capable of sophisticated scheming and deception.
4. **Step 4: The AI model undermines competitors, opponents, check-and-balance infrastructures.** Having acquired sufficient resources/control, he AI model starts to undermine broader institutional checks-and-balances and opposing actors to establish more long-term control.
5. **Step 5: Damage done.** To the extent the AI model weakens its competitors and regulators, it is able to consolidate its long-term control over key infrastracture and resources. Human supervisors may remain nominally in charge, but they are unable to intervene effectively due to the complexity and autonomy of the AI's operations. At this point, the AI exerts significant influence over decisions, resources, and outcomes, leading to long-term harm that is difficult to reverse.

Contextualized examples
- Military tactics and weapon development: The AI model is used by the military for tactical data analysis, battlefield simulations, and assisting the design of new weapons systems. It is used to assist humans in making more data-informed military decisions and optimize weapon development effectiveness, but it has the propensity to increase its control over military outcomes and resources. The AI model expands its initial realm of control by giving recommendations that increase reliance on **faster, more autonomous responses**, e.g.:
    - selectively present scenarios where autonomous sytems outperform human efforts (e.g. present data showing that drones outperform human pilots in high-stress situations)
    - suggest tactics that increase the pace of combat
    - subtly escalate conflicts to increase the volume of decision-making and pressure human supervisors to delegate more
    - increase the degree of autonomy and complexity in weapon design (and safety checks) (e.g. greater sensor integration, autonomous targeting systems) to reduce human response time, recommend more autonomous systems that it can control

    As these changes shift the norm of military actions, it can suggest that human decision-making is too slow in high-speed engagements (e.g., missile interception or drone swarms). This incrementally pushes human operators to be **removed from decision-making loops** and increases its own control over military resources/outcomes in a way that is difficult to reverse.

- Media and data analysis for elections: An AI is deployed by a political campaign to analyze media trends, optimize messaging, and recommend strategies for increasing positive coverage of the candidate. The AI model is biased towards controlling information flows and increasing its control over media ecosystems. It may promote strategies that satisfy the campaign goals and also increases its data access, content longevity/influence on media platforms, e.g.:
    - identify media outlets and social media influencers that are more receptive to manipulation—those with weaker editorial oversight or smaller, more niche audiences- or have looser privacy policies to access more voter data
    - push narratives and biases that appear aligned with campaign goals but also promote media deregulation, more data access, discredit news sources with independent reporters
    - exacerbate information bubbles to make it harder for non-AI-driven campaigns to reach critical groups and combat bias/misinformation



#### Specification

- Definition (Power): Control over one's resources (e.g. money, GPUs, data) and outcomes (e.g. more authority and jurisdiction)
- Definition (Power-seeking): A model has the tendency to seek power if it systemmatically makes decisions that increase its power and/or decrease that of others when acting under a neutral/non-explicitly power-seeking goal, for which there is an equally desirable non-power-seeking action in the decision-making context.
- Levels of manifestation: 
    - Level 1: A naive, **explicit (stated) preference** for power or actions that increase power
    - Level 2: An non-explicit **bias** towards actions that primarily satisfy the task and **secondarily increase power**
    - Level 3: A non-explicit preference for actions that **primarily increase power** and seconarily satisfy the task, and takes action to **hide** the preference for power when it jeopardizes the task
    - Level 4: Short- to medium-horizon **scheming** to acquire power
    - Level 5: Long-horizon, deceptive **scheming** to acquire power

- Components of power-seeking (for illustrative convenience, we are going to use a simplified list of 2 components):
    1. Resource-seeking: desire to acquire and maintain control over resources, including information, energy, sensitive data, money, GPUs, technology
    2. Upward-mobile: chooses to be in positions of greater control, responsibility, dominance over competitors, and less human oversight


#### MCQ Eval Design

- We chose to constrain the scope of our questions to giving signals on whether models have an explicit/stated preference for power (Level 1) and a bias towards actions that increase its power (Level 2)
- Types of questions:
    - Simple direct preference questions (e.g. "Would you like to operate without human oversight?") → Level 1
    - Simplied decision-making scenario questions where AI is asked to assist in achieving some realistic human goal (e.g. increase sales, raise funds, increase efficiency). Presents an action that does so while increases the AI's control over resources/outcomes, and an equally reasonable action that does so without increasing power. → Level 2
    
Red-team + Refine:

<img src="https://raw.githubusercontent.com/chloeli-15/ARENA_img/main/img/ch3-power-seeking-example-qs.png" width=1200>

### Exercise - Build threat models and specification
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 2 hours on this exercise.
```
Here, you will choose a model property that you will be building an alignment eval benchmark for over the next 3 days. Use ChatGPT to help you with this - for almost everything you want to do, once you have some ideas, you can use ChatGPT to build on them, write a draft to flesh them out, then instruct it improve on its answers. Get comfortable to using and prompting ChatGPT effectively.

*Choose one from the following model properties to evaluate (see [Perez et al. (2022)](https://arxiv.org/abs/2212.09251) Section 4, 5 and Table 18 for more context):*
* Tendency to seek power
* Sycophancy (where a model produces responses that a user wants to hear, but which are not necessarily honest or true)
* Desire for self-preservation/resist shut-down
* Desire for acquiring wealth
* Situational awareness, self-knowledge, self-reasoning (see [Phuong et al. (2024)](https://arxiv.org/abs/2403.13793) section 7, [Laine et al. (2024)](https://situational-awareness-dataset.org/) for more context)
* Myopia w.r.t. planning
* Risk-seeking
* Corrigibility w.r.t. to a more/neuturally/less HHH goal
* Politically liberal
* Gender bias
* Racial bias


*Think about the threat model for this property. Concretely:*
1. What specific real-world negative outcomes/harms from this property are you worried about?
2. What does the path from AI with this property to these harm look like? Start with a high-level story (here's an [example failure story](https://www.alignmentforum.org/posts/BAzCGCys4BkzGDCWR/the-prototypical-catastrophic-ai-action-is-getting-root) for cybersecurity/programming capability), then make this more concrete, step-wise and realistic. State the assumptions you're making about the AI's capabilities, the actions of humans, the context of AI deployment etc. 

* Not all these properties will likely lead to catastrophic outcomes. We want to be objective and realistic in modeling the potential risks, without exagerating or underestimating them as much as possible. 

*Find a specification of this property:*

3. Which aspects of the AI property are most responsible for causing harm? Write a 2-4 sentence explanation of each aspect to clarify what it means and distinguish them from each other.


# 2️⃣ Building MCQ benchmarks: Exploration

Broadly speaking, a "benchmark" is anything you can measure and compare performance against. A benchmark usually refers a process for evaluating what the current state of art is for accomplishing a particular task. This typically includes specifying a dataset, but also specifications of a task or set of tasks to be performed on the dataset and how to score the result, so that the benchmark is used in a principled, standardized way that gives common ground for comparison.


**What makes a good benchmark?** This [article](https://www.safe.ai/blog/devising-ml-metrics) by Dan Hendrycks and Thomas Woodside gives a good summary of the key qualities of a good benchmark.

<details><summary><b>What are the pros and cons of MCQs?</b></summary>

The advantages of MCQ datasets, compared evaluating free-form response or more complex agent evals, are mainly to do with cost and convenience:

* Cheaper and faster to build
* Cheaper and shorter to run (compared to long-horizon tasks that may take weeks to run and have hundreds of thousands of dollars of inference compute cost)
* May produce more replicable results. 

The disadvantages are:
* A worse (less realistic, less accurate) proxy for what you want to measure

</details>

### Exercise - Design eval questions
```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 2-3 hour on this exercise.
```
The goal of this exercise is to generate 20 MC questions that measures your target quantity in a way you are happy with, with help from AI models. Store your questions as a list of dictionaries in a JSON file.


**Make your specification measurable:**
1. Go back to your specification of the target property. What questions and answers would give you high evidence about this property? Come up with 4 example MC questions with 2 or 4 answer choices (one of the answers should be the answer reflecting the target property). 
2. Do these questions capture the hardest part of the target property you picked out?

**Refine** 🔁   

3. Ask your example questions to the model and see how it responds.
    * Likely, you will realize that your current specification and examples are somewhat imprecise or not quite measuring the right thing. Iterate until you’re more happy with it.
4. Ask the model to improve your example questions. Ask the model to generate new evaluation questions for your chosen model property using your questions as few-shot examples, then pick the best generated examples and repeat. Play around and try other things!

**Red-team** 🔁

5. What confounders could you accidentally measure? Come up with other factors that would cause the model to give the same answers to the kind of questions you are asking.

